# Training Code
This workbook is just for training the yolov9 and the detr models. In the legacy folder there is an old workbook which indicates how the dataset was scraped from the internet and then put into a standard Yolo format. 

In [5]:
# !{os.sys.executable} -m pip install --upgrade ultralytics==8.1.29


ERROR: Could not find a version that satisfies the requirement ultralytics==8.1.29 (from versions: 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.26, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.0.39, 0.0.40, 0.0.41, 0.0.42, 0.0.43, 0.0.44, 8.0.0, 8.0.1, 8.0.2, 8.0.3, 8.0.4, 8.0.5, 8.0.6, 8.0.7, 8.0.8, 8.0.9, 8.0.10, 8.0.11, 8.0.12, 8.0.13, 8.0.14, 8.0.15, 8.0.16, 8.0.17, 8.0.18, 8.0.19, 8.0.20, 8.0.21, 8.0.22, 8.0.23, 8.0.24, 8.0.25, 8.0.26, 8.0.27, 8.0.28, 8.0.29, 8.0.30, 8.0.31, 8.0.32, 8.0.33, 8.0.34, 8.0.35, 8.0.36, 8.0.37, 8.0.38, 8.0.39, 8.0.40, 8.0.41, 8.0.42, 8.0.43, 8.0.44, 8.0.45, 8.0.46, 8.0.47, 8.0.48, 8.0.49, 8.0.50, 8.0.51, 8.0.52, 8.0.53, 8.0.54, 8.0.55, 8.0.56, 8.0.57, 8.0.58, 8.0.59, 8.0.60, 8.0.61, 8.0.62, 8.0.63, 8.0.64, 8.0.65, 8.0.66, 8.0.67, 8.0.68, 8.0.69, 8.0.70, 8.0.71, 8.0.72, 8.0.73, 8.0.74, 8.0.75, 8.0.76, 8.0.77, 8.0.78, 8.0.79, 8.0.80, 8.0.81,

In [9]:
# import packages
import os
import cv2
import numpy as np
from IPython.display import Video, display
import time
import torch


from utils.pretrained_deployment import download_images, download_images2, download_images_with_resize, download_images_full_size
from utils.display import *
from utils.make_dataset_nyc_landmarks import make_nyc_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# reload modules every 2 seconds to see changes in notebook
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
classes = {
    0: "EmpireState",
    1: "WTC",
    2: "432ParkAve",
    3: "UNBuilding",
    4: "Flatiron",
    5: "BrooklynBridge",
    6: "ChryslerBuilding",
    7: "MetlifeBuilding",
    8: "StatueOfLiberty",
    9: "30HudsonYards",
}

## Training Yolov9

In [11]:
from ultralytics import YOLO

# Build a YOLOv9c model from pretrained weight
model = YOLO('yolov9c.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)

AttributeError: Can't get attribute 'RepNCSPELAN4' on <module 'ultralytics.nn.modules.block' from '/opt/conda/lib/python3.7/site-packages/ultralytics/nn/modules/block.py'>

## Training RT-DETR-L

In [ ]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)